Notes

This is going to be a logistic regression/ml classification model (supervised)

https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e

In [131]:
import pandas as pd
import numpy as np
from util import get_data, set_cwd_to_script, dependent_variable, categorical_variables, continuous_variables
set_cwd_to_script()
pd.set_option('display.max_rows', 500)

In [132]:
def filter_valid_outcomes(df):
    df = df[df["action_taken"].isin([1, 3])].copy()
    return df

def filter_null_columns(df, threshold=0.5):
    total_rows = len(df)
    for col in df.columns:
        total_nulls = df[col].isnull().sum()
        if total_nulls >= (threshold*total_rows):
            del df[col]
    return df

Correlation matrix

Independent variables that correlate strongly with the dependent variable (action taken) should be included in the model.

Alot of the independent variables are correlated with each other. This is called multicolinarity and can interfere with the model results.
https://towardsdatascience.com/multi-collinearity-in-regression-fe7a2c1467ea



In [133]:
loan_data = get_data("state_IL_application.csv")
loan_data = filter_valid_outcomes(loan_data)
loan_data = filter_null_columns(loan_data)
loan_data.isnull().sum()

activity_year                                    1
lei                                              0
derived_msa-md                                   0
state_code                                       0
county_code                                   1681
census_tract                                  1745
conforming_loan_limit                         2760
derived_loan_product_type                        0
derived_dwelling_category                        0
action_taken                                     0
purchaser_type                                   0
preapproval                                      0
loan_type                                        0
loan_purpose                                     0
lien_status                                      0
reverse_mortgage                                 0
open-end_line_of_credit                          0
business_or_commercial_purpose                   0
loan_amount                                      0
loan_to_value_ratio            

In [134]:
loan_data.shape

(536543, 54)

In [135]:
test_data = get_data("X_test.xlsx")

In [136]:
def process_categorical_variables(df, variable_list):
    # switch categories to integers for the model
    # set types as category
    for col in df:
        if col in variable_list:
            df[col] = df[col].fillna("not provided")
            df[col] = pd.Categorical(df[col])
            df[col] = df[col].cat.codes
            df[col] = df[col].astype("category")
    return df

def process_continuous_variables(df, variable_list):
    # standardize column data between 0 and 1
    for col in df:
        if col in variable_list:
            df[col] = df[col].replace({"Exempt": np.nan})
            # TODO look into an exempt/non exempt categorical variable
            df[col] = pd.to_numeric(df[col])
            df[col] = df[col].fillna(df[col].mean())
            max_value = df[col].max()
            min_value = df[col].min()
            df[col] = (df[col] - min_value) / (max_value - min_value)
    return df

loan_data = process_categorical_variables(loan_data, categorical_variables)
loan_data = process_continuous_variables(loan_data, continuous_variables)

In [137]:
loan_data.head()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,action_taken,...,initially_payable_to_institution,aus-1,denial_reason-1,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,0,373,9,0,46,1973,0,0,3,1.0,...,0,1,9,1.000000,0.2605,0.898453,0.363363,1.000000,1.000000,0.157895
1,0,129,16,0,49,2148,0,0,3,1.0,...,3,6,10,0.177224,0.1381,0.636364,0.297297,0.209798,0.251380,0.776316
4,0,540,5,0,21,1628,1,0,3,1.0,...,0,5,9,0.121950,0.0920,0.820116,0.855856,0.120505,0.124540,0.763158
5,0,998,5,0,15,1362,1,0,3,1.0,...,0,4,9,0.143741,0.1483,0.820116,0.828829,0.126611,0.144549,0.539474
8,0,1001,14,0,81,2642,0,5,3,1.0,...,0,0,9,0.239448,0.2370,0.798839,0.504505,0.262315,0.247700,0.250000


In [138]:
corr=loan_data.corr()
corr.style.background_gradient(cmap='coolwarm')

,action_taken,loan_amount,interest_rate,rate_spread,total_loan_costs,origination_charges,loan_term,property_value,income,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
action_taken,1.000000,-0.038320,-0.000000,0.000000,0.000000,-0.000000,0.017630,-0.020142,-0.044514,-0.028816,0.114200,-0.031732,-0.108461,-0.047171,-0.026888,0.050283
loan_amount,-0.038320,1.000000,-0.004911,0.028593,0.001704,0.022231,0.020042,0.797476,0.106918,-0.008151,-0.009769,0.052522,0.125118,-0.012341,-0.040818,0.010784
interest_rate,-0.000000,-0.004911,1.000000,0.014644,0.000270,0.000523,-0.004241,-0.002346,-0.004126,-0.007542,0.017876,-0.009123,-0.020790,-0.010904,-0.006882,0.014482
rate_spread,0.000000,0.028593,0.014644,1.000000,0.001618,0.012477,0.003017,0.026722,-0.009081,-0.001265,0.011474,-0.008495,-0.020512,-0.003973,0.000535,0.006491
total_loan_costs,0.000000,0.001704,0.000270,0.001618,1.000000,0.030869,0.013425,-0.000225,-0.001016,0.001778,0.008773,0.005874,-0.005226,-0.000465,0.000179,0.001235
origination_charges,-0.000000,0.022231,0.000523,0.012477,0.030869,1.000000,0.051312,0.011029,0.005566,0.012257,0.063651,0.047631,0.003585,-0.000781,-0.009671,0.007936
loan_term,0.017630,0.020042,-0.004241,0.003017,0.013425,0.051312,1.000000,-0.017601,-0.022351,0.010730,0.041862,0.062050,0.039696,-0.000803,-0.008485,0.027457
property_value,-0.020142,0.797476,-0.002346,0.026722,-0.000225,0.011029,-0.017601,1.000000,0.111651,-0.017204,-0.021241,0.038162,0.134281,-0.018949,-0.049196,0.018185
income,-0.044514,0.106918,-0.004126,-0.009081,-0.001016,0.005566,-0.022351,0.111651,1.000000,-0.009717,-0.055011,0.060531,0.209224,-0.001221,-0.040695,-0.002256
tract_population,-0.028816,-0.008151,-0.007542,-0.001265,0.001778,0.012257,0.010730,-0.017204,-0.009717,1.000000,0.002299,0.227417,0.073669,0.927499,0.876582,-0.461116


In [139]:
corr["absolute_correlation"] = corr["action_taken"].abs()
corr = corr.sort_values(by=["absolute_correlation"], ascending=False)
corr["absolute_correlation"]

action_taken                         1.000000e+00
tract_minority_population_percent    1.142002e-01
tract_to_msa_income_percentage       1.084611e-01
tract_median_age_of_housing_units    5.028340e-02
tract_owner_occupied_units           4.717094e-02
income                               4.451406e-02
loan_amount                          3.832022e-02
ffiec_msa_md_median_family_income    3.173155e-02
tract_population                     2.881565e-02
tract_one_to_four_family_homes       2.688807e-02
property_value                       2.014189e-02
loan_term                            1.762958e-02
interest_rate                        5.642070e-14
origination_charges                  1.010659e-14
total_loan_costs                     2.609016e-15
rate_spread                          1.692726e-15
Name: absolute_correlation, dtype: float64

In [140]:
loan_data.shape

(536543, 54)